In [27]:
from pathlib import Path
from matplotlib.style import use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters


import torch
import torch.nn as nn

dataDir = Path.cwd().parent.parent/'Data/processed/ts_targetCows/'
usecols = ['id', 'MilkingEventDateTime', 'FarmName_Pseudo', 'TrafficDeviceName', 'MilkProduction', 'timeDelta_Seconds', 'LactationNumber', 'DaysInMilk', 'BreedName', 'Age']
threshold_time = 800

# labeling cow with good/bad learner(0/1)
def labeling_data(threshold_time, cow_total):
    global learner
    timeCost = cow_total.timeDelta_Seconds.mean()
    if timeCost > threshold_time:
        learner = 1 # good learner
    else:
        learner = 0 # bad learner
    cow_total['label'] = learner
    return cow_total

In [28]:
# integrate all the cows data into one dataset
filelist = list(Path(dataDir).glob('*.csv'))
for i, _ in enumerate(filelist):
    fileName = 'cow_' + str(i) + '.csv'
    single_cow = pd.read_csv(dataDir/fileName, encoding='utf-8', usecols=usecols)
    single_cow = labeling_data(threshold_time, single_cow)
    single_cow.sort_values(by=['MilkingEventDateTime'], inplace=True)
    if i == 0:
        cow_total = single_cow
    cow_total = pd.concat([cow_total, single_cow])

In [62]:
#df_static_features = cow_total[['FarmName_Pseudo']]
static_cols = ['FarmName_Pseudo', 'TrafficDeviceName', 'LactationNumber', 'BreedName']
timeSeries_cols = ['DaysInMilk']
all_timeSeries_cols = ['Age', 'MilkProduction', 'timeDelta_Seconds', 'DaysInMilk']
output_col = ['label']


In [73]:
cow_label = cow_total[['id', 'label']].copy()
cow_timeseries = cow_total[['id', 'MilkingEventDateTime']].copy()
cow_timeseries.index = range(len(cow_timeseries))
ts_processed = cow_total[timeSeries_cols].copy()
# normalize numerical features
scaler_minmax = MinMaxScaler()
scaler_std = StandardScaler()
ts_processed = pd.DataFrame(scaler_std.fit_transform(ts_processed))
cow_timeseries = pd.concat([cow_timeseries, ts_processed], axis=1)
cow_timeseries.rename(columns={0: 'DaysInMilk'}, inplace=True)
# fetch y for feature extraction
y = cow_label.drop_duplicates(subset=['id'])
y = y["label"]
y.index = range(1,len(y)+1) 

In [66]:
cow_timeseries

,id,MilkingEventDateTime,DaysInMilk
0,1,2021-07-05 11:02:14,-1.670539
1,1,2021-07-05 20:52:47,-1.670539
2,1,2021-07-06 06:41:09,-1.659383
3,1,2021-07-06 16:25:11,-1.659383
4,1,2021-07-07 01:39:16,-1.648226
...,...,...,...
13983,16,2022-08-06 06:38:20,2.133740
13984,16,2022-08-06 14:50:23,2.133740
13985,16,2022-08-07 01:52:24,2.144896
13986,16,2022-08-07 11:38:29,2.144896


In [67]:
extracted_features = extract_features(cow_timeseries, column_id="id", column_sort="MilkingEventDateTime")

Feature Extraction: 100%|██████████| 16/16 [00:07<00:00,  2.09it/s]


In [68]:
impute(extracted_features)
features_filtered = select_features(extracted_features, y)

d:\Toolbox\python\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['DaysInMilk__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  warnings.warn(


In [37]:
ts_extracted_dataset = cow_total[['id']].copy()
ts_extracted_dataset.drop_duplicates(subset=['id'], inplace=True)
ts_extracted_dataset.index = range(1, len(ts_extracted_dataset)+1)

In [69]:
ts_extracted_dataset = pd.concat([ts_extracted_dataset, features_filtered], axis=1)

In [75]:
ts_extracted_dataset = pd.concat([ts_extracted_dataset, y], axis=1)

In [78]:
ts_extracted_dataset.to_csv(dataDir/"ts_extracted_dataset.csv")
ts_extracted_dataset

,id,timeDelta__sum_values,timeDelta__ar_coefficient__coeff_0__k_10,"timeDelta__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0","timeDelta__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0","timeDelta__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0","timeDelta__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0","timeDelta__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.2","timeDelta__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.2","timeDelta__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",...,"Age__fft_coefficient__attr_""real""__coeff_0",Age__count_above__t_0,Age__count_below__t_0,"Age__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.4",Age__quantile__q_0.7,"Age__fft_coefficient__attr_""angle""__coeff_0",Age__mean_abs_change,Age__mean,Age__variation_coefficient,label
1,1,-258.270455,-0.059432,0.028329,0.083289,0.021393,0.331732,0.003371,0.026488,0.079550,...,-258.270455,0.200708,0.799292,0.013697,-0.226190,180.0,0.331732,-0.152462,-5.832477,0
2,2,-65.818250,-0.052343,0.092498,0.217239,0.045306,0.769818,0.005436,0.099213,0.236574,...,-65.818250,0.261468,0.738532,0.074653,-0.122847,180.0,0.769818,-0.075480,-12.314639,0
3,3,222.515677,0.224628,0.243725,0.393940,0.088553,1.092149,0.015533,0.202925,0.358721,...,222.515677,0.436019,0.563981,0.153784,0.362611,0.0,1.092149,0.351526,3.822593,1
4,4,191.531087,0.421148,0.323197,0.454690,0.117113,1.072078,0.015992,0.253044,0.403896,...,191.531087,0.470054,0.529946,0.193257,0.455690,0.0,1.072078,0.347606,3.096461,1
5,5,-15.281514,-0.015041,0.080499,0.225830,0.029538,0.680120,0.004516,0.062471,0.199104,...,-15.281514,0.258065,0.741935,0.051094,-0.074841,180.0,0.680120,-0.016998,-50.369007,0
6,6,79.167425,0.044996,0.168901,0.317342,0.068374,0.871197,0.011733,0.170684,0.331177,...,79.167425,0.353096,0.646904,0.133740,0.125825,0.0,0.871197,0.104305,10.689828,1
7,7,-127.520034,-0.106161,0.087597,0.220981,0.038765,0.643985,0.006356,0.089296,0.234857,...,-127.520034,0.247845,0.752155,0.061424,-0.103166,180.0,0.643985,-0.137414,-5.414806,0
8,8,276.115971,0.167051,0.466056,0.511193,0.204795,1.229253,0.022964,0.505636,0.566458,...,276.115971,0.413450,0.586550,0.436364,0.581029,0.0,1.229253,0.343856,3.989830,1
9,9,-75.125493,-0.074791,0.089882,0.231704,0.036224,0.634169,0.006688,0.082267,0.229354,...,-75.125493,0.255629,0.744371,0.057030,-0.091049,180.0,0.634169,-0.099504,-7.987953,0
10,10,36.089554,0.021588,0.140618,0.274528,0.065268,0.889185,0.006735,0.152795,0.302684,...,36.089554,0.288241,0.711759,0.129388,-0.039030,0.0,0.889185,0.037554,28.039183,1


In [71]:
from tsfresh.feature_selection.relevance import calculate_relevance_table
rt = calculate_relevance_table(extracted_features, y)
rt

,feature,type,p_value,relevant
feature,,,,
DaysInMilk__first_location_of_maximum,DaysInMilk__first_location_of_maximum,real,0.001088,False
DaysInMilk__has_duplicate_max,DaysInMilk__has_duplicate_max,binary,0.001399,False
"DaysInMilk__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.8","DaysInMilk__change_quantiles__f_agg_""mean""__is...",real,0.006993,False
"DaysInMilk__change_quantiles__f_agg_""var""__isabs_True__qh_1.0__ql_0.6","DaysInMilk__change_quantiles__f_agg_""var""__isa...",real,0.006993,False
"DaysInMilk__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.8","DaysInMilk__change_quantiles__f_agg_""mean""__is...",real,0.006993,False
...,...,...,...,...
DaysInMilk__permutation_entropy__dimension_4__tau_1,DaysInMilk__permutation_entropy__dimension_4__...,constant,NaN,False
DaysInMilk__permutation_entropy__dimension_5__tau_1,DaysInMilk__permutation_entropy__dimension_5__...,constant,NaN,False
DaysInMilk__permutation_entropy__dimension_6__tau_1,DaysInMilk__permutation_entropy__dimension_6__...,constant,NaN,False


In [8]:
# one-hot encode categorical features
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
cat = ohe.fit_transform(cow_total[categorical_cols])
categorical_data = torch.tensor(cat, dtype=torch.float)
# normalize numerical features
scaler = MinMaxScaler()
num = scaler.fit_transform(cow_total[numerical_cols])
numerical_data = torch.tensor(num, dtype=torch.float64)
# convert label to tensor
output = torch.tensor(cow_total['label'])
train_data = torch.cat((numerical_data, categorical_data), 1)

# split training set and test set
df_data = pd.DataFrame(train_data) 
df_ouput = pd.DataFrame(output)
df_dataset = pd.concat([df_data, df_ouput], axis=1)
df_dataset.dropna(inplace=True)
df_dataset.to_csv(dataDir.parent/'cow_dataset.csv', index=False)


In [7]:
#cow_total.dropna
print(cow_total.id.unique())
cow_total

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


,FarmName_Pseudo,MilkingEventDateTime,TrafficDeviceName,MilkProduction,timeDelta_Seconds,LactationNumber,DaysInMilk,BreedName,Age,id,label
0,a624fb9a,2021-07-05 11:02:14,vms2,9.07,6215.0,1.0,2.0,1,1.86,1,0
1,a624fb9a,2021-07-05 20:52:47,vms1,7.67,79.0,1.0,2.0,1,1.86,1,0
2,a624fb9a,2021-07-06 06:41:09,vms2,7.12,392.0,1.0,3.0,1,1.86,1,0
3,a624fb9a,2021-07-06 16:25:11,vms2,7.31,43.0,1.0,3.0,1,1.86,1,0
4,a624fb9a,2021-07-07 01:39:16,vms2,4.30,232.0,1.0,4.0,1,1.87,1,0
...,...,...,...,...,...,...,...,...,...,...,...
902,a624fb9a,2022-08-06 06:38:20,vms2,8.97,143.0,1.0,343.0,1,3.37,16,0
903,a624fb9a,2022-08-06 14:50:23,vms2,6.23,114.0,1.0,343.0,1,3.37,16,0
904,a624fb9a,2022-08-07 01:52:24,vms1,9.98,1192.0,1.0,344.0,1,3.37,16,0
905,a624fb9a,2022-08-07 11:38:29,vms1,7.98,1071.0,1.0,344.0,1,3.37,16,0


In [17]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()